In [1]:
import time
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.optim as optim
import torchvision
from torchvision.models.efficientnet import MBConvConfig, FusedMBConvConfig

import optuna

sys.path.append("/jet/home/azhang19/stat 214/stat-214-lab2-group6/code/modeling")
from preprocessing import to_NCHW, pad_to_384x384, standardize_images
from autoencoder import EfficientNetEncoder, EfficientNetDecoder, AutoencoderConfig

device = "cuda" if torch.cuda.is_available() else "cpu"

torch.set_float32_matmul_precision('high')
torch.backends.cudnn.benchmark = True

use_amp = True

/jet/home/azhang19/.conda/envs/env_214/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load and preprocess data
data = np.load("/jet/home/azhang19/stat 214/stat-214-lab2-group6/data/array_data.npz")
unlabeled_images, unlabeled_masks, labeled_images, labeled_masks, labels = data["unlabeled_images"], data["unlabeled_masks"], data["labeled_images"], data["labeled_masks"], data["labels"]

unlabeled_images = pad_to_384x384(to_NCHW(unlabeled_images))
unlabeled_masks = pad_to_384x384(unlabeled_masks)

labeled_images = pad_to_384x384(to_NCHW(labeled_images))
labeled_masks = pad_to_384x384(labeled_masks)
labels = pad_to_384x384(labels)

# Convert to tensors and move to GPU
unlabeled_images = torch.tensor(unlabeled_images, dtype=torch.float32).to(device)  # [161, 8, 384, 384]
unlabeled_masks = torch.tensor(unlabeled_masks, dtype=torch.bool).to(device)    # [161, 384, 384]

labeled_images = torch.tensor(labeled_images, dtype=torch.float32).to(device)      # [3, 8, 384, 384]
labeled_masks = torch.tensor(labeled_masks, dtype=torch.bool).to(device)        # [3, 384, 384]
labels = torch.tensor(labels, dtype=torch.long).to(device)                      # [3, 384, 384]


# Standardize images
unlabeled_images, std_channel, mean_channel = standardize_images(unlabeled_images, unlabeled_masks)
labeled_images, _, _ = standardize_images(labeled_images, labeled_masks, std_channel, mean_channel)

In [3]:
encoder_config = [
    FusedMBConvConfig(1, 3, 1, 16, 16, 1),  # 384x384x8 -> 384x384x16
    FusedMBConvConfig(4, 3, 2, 16, 32, 1),  # 384x384x16 -> 192x192x32
    MBConvConfig(4, 3, 2, 32, 64, 1),       # 192x192x32 -> 96x96x64
]

# Build encoder and decoder
encoder = EfficientNetEncoder(
    inverted_residual_setting=encoder_config,
    dropout=0.1,
    input_channels=8,
    last_channel=64,
)

decoder = EfficientNetDecoder()

autoencoder = nn.Sequential(encoder, decoder).train().to(device)
autoencoder.load_state_dict(torch.load(f"/jet/home/azhang19/stat 214/stat-214-lab2-group6/code/modeling/ckpt/AutoencoderConfig([1, 1, 1], flip=True, rotate=True)/autoencoder_12800.pth"))

encoder = encoder.eval()
with torch.inference_mode():
    feature = encoder(labeled_images)
    feature = nn.functional.interpolate(feature, size=384, mode="bicubic", antialias=True)

In [9]:
def train_and_validate(
    train_data, train_labels, val_data, val_labels,
    kernel_size, epochs, lr, weight_decay, optimizer_class, loss_fn, l1, device
):
    # Create the classifier
    classifier = nn.Conv2d(64, 1, kernel_size=kernel_size, 
                          padding="same", padding_mode="replicate").to(device)
    classifier.train()

    # Instantiate the optimizer
    optimizer = optimizer_class(classifier.parameters(), lr=lr, weight_decay=weight_decay)

    # Training loop
    for epoch in range(epochs):
        classifier.train()
        optimizer.zero_grad(set_to_none=True)
        pred = classifier(train_data)
        loss, acc, f1 = loss_fn(pred, train_labels)
        # Add L1 regularization
        loss = loss + l1 * l1_reg(classifier)
        loss.backward()
        optimizer.step()

    # Validation
    classifier.eval()
    with torch.inference_mode():
        val_pred = classifier(val_data)
        val_loss, val_acc, val_f1 = loss_fn(val_pred, val_labels)

    return val_f1

# Apply torch.compile for optimization (if supported)
train_and_validate = torch.compile(train_and_validate)

def objective(trial):
    # Suggest hyperparameters with updated API calls
    epochs = trial.suggest_int("epochs", 200, 600)
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["SGD", "AdamW"])
    kernel_size = trial.suggest_categorical("kernel_size", [1, 2, 3])
    loss_name = trial.suggest_categorical("loss_fn", ["bce", "soft_margin"])
    l1 = trial.suggest_float("l1", 1e-5, 1e-1, log=True)
    
    # Map string to actual optimizer class
    optimizer_class = torch.optim.SGD if optimizer_name == "SGD" else torch.optim.AdamW

    # Map string to loss function (assumes these are defined)
    loss_fn = masked_bce_loss_acc if loss_name == "bce" else masked_bce_loss_acc

    # Cross-validation indices (modify as needed)
    train_val_idx = [0, 1]
    
    # Container for metrics from each fold
    fold_records = torch.zeros(len(train_val_idx))

    # Assuming feature and labels are defined globally (e.g., torch tensors)
    for i in train_val_idx:
        # Leave-one-out style split
        train_idx = [j for j in train_val_idx if j != i]
        val_idx = [i]

        # Get training and validation data
        train_data = feature[train_idx]
        train_labels = labels[train_idx]
        val_data = feature[val_idx]
        val_labels = labels[val_idx]

        # Train and validate
        val_f1 = train_and_validate(
            train_data=train_data,
            train_labels=train_labels,
            val_data=val_data,
            val_labels=val_labels,
            kernel_size=kernel_size,
            epochs=epochs,
            lr=lr,
            weight_decay=weight_decay,
            optimizer_class=optimizer_class,
            loss_fn=loss_fn,
            l1=l1,
            device=device  # Assumes device is defined globally (e.g., 'cuda')
        )

        fold_records[i] = val_f1
    
    # Return average F1 score across folds
    return fold_records.mean().item()

In [ ]:
study = optuna.create_study(direction="maximize")

# Optimize the study by running a number of trials (e.g., 100 trials).
study.optimize(objective, n_trials=100)

[I 2025-03-08 03:49:25,238] A new study created in memory with name: no-name-0dfe4a55-dfa8-45a4-9b6e-913dc081c328
W0308 03:49:29.497000 33388 site-packages/torch/_logging/_internal.py:1089] [12/0] Profiler function <class 'torch.autograd.profiler.record_function'> will be ignored
('Grad tensors ["L['self'].param_groups[0]['params'][0].grad", "L['self'].param_groups[0]['params'][1].grad"] will be copied during cudagraphs execution.If using cudagraphs and the grad tensor addresses will be the same across runs, use torch._dynamo.decorators.mark_static_address to elide this copy.',)
('Grad tensors ["L['self'].param_groups[0]['params'][0].grad", "L['self'].param_groups[0]['params'][1].grad"] will be copied during cudagraphs execution.If using cudagraphs and the grad tensor addresses will be the same across runs, use torch._dynamo.decorators.mark_static_address to elide this copy.',)
[I 2025-03-08 03:49:33,369] Trial 0 finished with value: 0.7514251470565796 and parameters: {'epochs': 241, '

[W 2025-03-08 03:50:45,720] Trial 59 failed with parameters: {'epochs': 599, 'lr': 0.0026581942837536543, 'weight_decay': 0.016468585255105837, 'optimizer': 'AdamW', 'kernel_size': 2, 'loss_fn': 'soft_margin', 'l1': 0.011467467047446327} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/jet/home/azhang19/.conda/envs/env_214/lib/python3.13/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/tmp/ipykernel_33388/665618713.py", line 70, in objective
    val_f1 = train_and_validate(
        train_data=train_data,
    ...<10 lines>...
        device=device  # Assumes device is defined globally (e.g., 'cuda')
    )
  File "/jet/home/azhang19/.conda/envs/env_214/lib/python3.13/site-packages/torch/_dynamo/eval_frame.py", line 574, in _fn
    return fn(*args, **kwargs)
  File "/var/tmp/ipykernel_33388/665618713.py", line 6, in train_and_validate
    classifier = nn.Conv2d(64, 1, kernel_siz

KeyboardInterrupt: 

In [9]:
# Print out the best trial.
print("Best trial:")
best_trial = study.best_trial
print("  Best Validation F1 Score: {:.4f}".format(best_trial.value))
print("  Hyperparameters:")
for param_name, param_value in best_trial.params.items():
    print("    {}: {}".format(param_name, param_value))

Best trial:
  Best Validation F1 Score: 0.8262
  Hyperparameters:
    epochs: 325
    lr: 0.0003618543621266028
    weight_decay: 1.979201053287094e-05
    optimizer: SGD
    kernel_size: 3
    loss_fn: bce
    l1: 0.0047670779365748895
